# Introduction

In this notebook, we demonstrate the steps needed create a model for predicting Remaining Useful Life for turbofan engines based on data collected by devices and routed into storage via the IoT Hub. The notebook assumes that you have complete the device data generation steps from the [IoT  Edge for Machine Learning](aka.ms/IoTEdgeMLPaper). The data generated from the devices needs to be in an Azure Storage account blob container in the same Azure Subscription as you will use to create the Azure Machine Learning service workspace using this notebook.  

The steps we will complete in this notebooks are:
   1. Create a Machine Learning service workspace for managing the experiments, compute, and models for this sample
   1. Load training data from Azure Storage
   1. Prepare the data for training the model
   1. Explore the data 
   1. Remotely train the model
   1. Test the model using test data

The intent is not to provide an extensive coverage of machine learning in Azure as that is covered in much depth elsewhere [here for example](https://github.com/Azure/MachineLearningNotebooks), but to demonstrate how machine learning can be used with IoT Edge.

# Setup notebook

Please ensure that you are running the notebook under the Python 3.6 kernal. Intall fastavro and setup interactive shell to display output nicely.

In [26]:
!pip install fastavro

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

    100% |████████████████████████████████| 1.2MB 11.6MB/s ta 0:00:01


# Set global properties

These properties will be used throughout the notebook.
   * `AZURE_SUBSCRIPTION_ID` - the Azure subscription containing the storage account where device data has been uploaded. We will create the Machine Learning service workspace (ml workspace) in this subscription.
   * `ML_WORKSPACE_NAME`  name to give the ml workspace
   * `AZURE_IOT_HUB_NAME` - name of the Azure IoT Hub used in creating the device data using the DeviceHarness.  See [IoT  Edge for Machine Learning](aka.ms/IoTEdgeMLPaper) for details.
   * `STORAGE_ACCOUNT_NAME` - name of the Azure Storage account where device data was routed via IoT Hub.
   * `STORAGE_ACCOUNT_KEY` - access key for the Azure Storage account
   * `STORAGE_ACCOUNT_CONTAINER` - name of Azure Storage blob container where device data was routed via IoT Hub.

In [ ]:
AZURE_SUBSCRIPTION_ID = ''
ML_WORKSPACE_NAME = ''
AZURE_IOT_HUB_NAME = ''
STORAGE_ACCOUNT_NAME= ''
STORAGE_ACCOUNT_KEY= ''
STORAGE_ACCOUNT_CONTAINER= ''

# Create a workspace

## What is an Azure ML Workspace and Why Do I Need One?

An Azure ML workspace is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows. In particular, an Azure ML workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, operationalization, and the monitoring of operationalized models.

In addition to creating the workspace, the cell below writes a file, config.json, to a ./aml_config/config.json, which allows the Workspace object to be reloaded later.




><font color=gray>_Note: currently Workspaces are supported in the following regions: eastus2, eastus,westcentralus, southeastasia, westeurope, australiaeast, westus2, southcentralus_</font>

In [ ]:
from azureml.core import Workspace
workspace_name = ML_WORKSPACE_NAME
subscription_id = AZURE_SUBSCRIPTION_ID
resource_group = workspace_name

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,    
                      resource_group=resource_group,
                      create_resource_group=True,
                      location='westus2' # or other supported Azure region
                     )

ws.write_config(path='./aml_config')

## Workspace details

If there is no workspace object in the notebook, the  `Workspace.from_config()` reads the file **aml_config/config.json** and loads the detail. It then prints the Workspace details.

In [ ]:
import azureml.core
import pandas as pd
from azureml.core import Workspace

if 'ws' not in globals():
    ws = Workspace.from_config()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

# Download data from storage

The first step toward creating a model for RUL is to explore the data and understand its shape. We will download the data for this purpose, realizing that in the case of larger data sets only a sample of the data would be used at this step.

## Register storage account

The Datastore is a convenient construct associated the Workspace to upload/download data, and interact with it from remote compute targets. Register the Azure Storage account and container where device data was routed by IoT Hub using the information about the storage container provided at the beginning of the notebook.


In [ ]:
from azureml.core import Datastore

ds = Datastore.register_azure_blob_container(workspace=ws,
                                             datastore_name='turbofan',
                                             container_name=STORAGE_ACCOUNT_CONTAINER,
                                             account_name=STORAGE_ACCOUNT_NAME,
                                             account_key=STORAGE_ACCOUNT_KEY,
                                             create_if_not_exists=False)

## Use Datastore to download data

Use the Datastore to download the files to the local machine. The prefix is the top level path to download, which should be the name of the IoT Hub. 

In [ ]:
ds.download(target_path="./data/download", prefix=AZURE_IOT_HUB_NAME)

# Load train data

The data we just downloaded represent a series of messages sent by each device stored in [Apache Avro](https://avro.apache.org/docs/current/)(avro) format. We will use the fastavro package to deserialize the records from the avro files.
Here is an example deserialized record from the avro file. 

```json 
{
    "EnqueuedTimeUtc": "2018-12-01T01: 16: 22.0950000Z",
    "Properties": {},
    "SystemProperties": {
        "connectionDeviceId": "Client_3",
        "connectionAuthMethod": {
            "scope": "device",
            "type": "sas",
            "issuer": "iothub",
            "acceptingIpFilterRule": null
        },
        "connectionDeviceGenerationId": "636791290544434625",
        "contentType": "application/json",
        "contentEncoding": "utf-8",
        "enqueuedTime": "2018-12-01T01: 16: 22.0950000Z"
    },
    "Body": b'{
        "CycleTime": 1,
        "OperationalSetting1": -0.0001,
        "OperationalSetting2": 0.0001,
        "OperationalSetting3": 100.0,
        "Sensor1": 518.67,
        "Sensor2": 642.03,
       //Sensor 3-19 ommitted for brevity
        "Sensor20": 38.99,
        "Sensor21": 23.296
    }
}```

Taken together the messages represent a time series of data for multiple engines. Each engine is operating normally at the start of each time series, and develops a fault at some point during the series. The fault grows in magnitude until system failure (i.e. the failure point for the engine is the final cycle in the set). The remaining useful life (RUL) is therefore expressed as: 

$$RUL_{current} = Cycle_{max} - Cycle_{current}$$


## Create utils for loading data from avro files

Define a set of utility methods for loading the data from the avro files. We use thes utilities to load the locally downloaded data. Later in the notebook, these same utilities will form the basis of data processing for remote training (see **Train regression using Azure AutoMl and remote execution** below)

In [ ]:
%%writefile ./utils.py

import glob
import json
import pandas as pd

from fastavro import reader
from os.path import isfile

# parse connectionDeviceId and return the int part
# (e.g. Client_1 becomes 1)
def get_unit_num (unit_record):
    unit = unit_record["connectionDeviceId"]
    return int(unit.split('_')[1])

# create data row from avro file record
def load_cycle_row(record):
    json_body = record["Body"].decode()
    row = json.loads(json_body)
    row.update({'Unit': get_unit_num(record["SystemProperties"])})
    row.update({'QueueTime': pd.to_datetime(record["EnqueuedTimeUtc"])})
    return row

# add row to data frame
def append_df(base_df, append_df):
    if(base_df is None):
        base_df = pd.DataFrame(append_df)
    else:
        base_df = base_df.append(append_df, ignore_index=True)
    return base_df

# sort rows and columns in dataframe
def sort_and_index(index_data):
    #sort rows and reset index
    index_data.sort_values(by=['Unit', 'CycleTime'], inplace=True)
    index_data.reset_index(drop=True, inplace=True)
    
    #fix up column sorting for convenience in notebook
    sorted_cols = (["Unit","CycleTime", "QueueTime"] 
                   + ["OperationalSetting"+str(i) for i in range(1,4)] 
                   + ["Sensor"+str(i) for i in range(1,22)])

    return index_data[sorted_cols]

# load data from an avro file and return a dataframe
def load_avro_file(avro_file_name):
    with open(avro_file_name, 'rb') as fo:
        file_df = None
        avro_reader = reader(fo)
        print ("load records from file: %s" % avro_file_name)
        for record in avro_reader:
            row = load_cycle_row(record)
            file_df = append_df(base_df=file_df, append_df=[row])
        return file_df

# load data from all avro files in given dir 
def load_avro_directory(avro_dir_name):
    dir_df = None
    for file_name in glob.iglob(avro_dir_name, recursive=True):
        if isfile(file_name):
            file_df = load_avro_file(file_name)
            dir_df = append_df(base_df=dir_df, append_df=file_df)
    print("loaded %d records" % dir_df.shape[0])
    return sort_and_index(dir_df)

# add max cycle to each row in the data
def add_maxcycle(data_frame):
    # cleanup column if it already exists
    if 'MaxCycle' in data_frame.columns:
        data_frame.drop('MaxCycle', axis=1, inplace=True)

    total_cycles = data_frame.groupby(['Unit']).agg({'CycleTime' : 'max'}).reset_index()
    total_cycles.rename(columns = {'CycleTime' : 'MaxCycle'}, inplace = True)
    return data_frame.merge(total_cycles, how = 'left', left_on = 'Unit', right_on = 'Unit')

# return a remaining useful life class based on RUL
def classify_rul(rul):
     if (rul <= 25):
          return 'F25'
     elif (rul <= 75):
          return 'F75'
     elif (rul <= 150):
          return 'F150'
     else:
          return 'Full'
    
# add remaining useful life and remaing useful life class
# to each row in the data
def add_rul(data_frame):
    data_frame = add_maxcycle(data_frame)
    
    if 'RUL' in data_frame.columns:
        data_frame.drop('RUL', axis=1, inplace=True)
    data_frame['RUL'] = data_frame.apply(lambda r: int(r['MaxCycle'] - r['CycleTime']), axis = 1)

    if 'RulClass' in data_frame.columns:
        data_frame.drop('RulClass', axis=1, inplace=True)
    data_frame['RulClass'] = data_frame.apply(lambda r: classify_rul(r['RUL']), axis = 1)
    
    return data_frame



## Use utils to load data from download directory

In [ ]:
import utils

train_pd =  utils.load_avro_directory('./data/download/**/*')

train_pd.head(5)

## Calculate remaining useful life and RUL class labels

Add RUL for regression training and RulClass for classification

In [ ]:
train_pd = utils.add_rul(train_pd)

cols = ['Unit', 'CycleTime', 'MaxCycle', 'RUL', 'RulClass']
#show first 5 rows
train_pd[cols].head(5)

#show last 5 rows for engine 3
train_pd[train_pd['Unit'] == 3][cols].tail(5)

## Display train data

In [ ]:
train_pd.head(5)

# Explore the data

Visualize the data to start to get a sense of how features like sensor measurements and operations settings relate to remaining useful life (RUL)

## Sensor readings and RUL

Create a scatterplot for each sensor measurement vs. RUL. Notice that some measurements (e.g. sensor 2) seem to be correlated strongly to RUL whereas other measurements (e.g. sensor 1) stay constant throughout the life of the engine.

    
><font color=gray>_Note: the data is limited to the first 10 engine units for speed of rendering_</font>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#select the data to plot
plotData = train_pd.query('Unit < 10');

sns.set()
g = sns.PairGrid(data=plotData,
                x_vars = ['RUL'],
                y_vars = ["Sensor"+str(i) for i in range(1,22)],
                hue="Unit",
                height=3,
                aspect=2.5,
                palette="Paired")
g = g.map(plt.scatter, alpha=0.3)
g = g.set(xlim=(300,0))
g = g.add_legend()


## Operational settings and RUL

Create a scatterplot for each operation setting vs. RUL. Operational settings do not seem to correlate with RUL.
    
><font color=gray>_Note: the data is limited to the first 10 engine units for speed of rendering_</font>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import utils

plotData = train_pd.query('Unit < 10');
sns.set()
g = sns.PairGrid(data=plotData,
                x_vars = ['RUL'],
                y_vars = ["OperationalSetting"+str(i) for i in range(1,4)],
                hue="Unit",
                size=3,
                aspect=2.5,
                palette="Paired")
g = g.map(plt.scatter, alpha=0.3)
g = g.set(xlim=(300,0))
g = g.add_legend()


# Train model using Azure AutoMl and remote execution

In this section, we will use the Azure Machine Learning service to build a model to predict remaining useful life.

##  Create remote compute target

Azure ML Managed Compute is a managed service that enables data scientists to train machine learning models on clusters of Azure virtual machines, including VMs with GPU support. This code creates an Azure Managed Compute cluster if it does not already exist in your workspace. 

 **Creation of the cluster takes approximately 5 minutes.** If the cluster is already in the workspace this code uses it and skips the creation process.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

CLUSTER_NAME = "mlturbo"

# choose a name for your cluster
batchai_cluster_name = CLUSTER_NAME + "gpu"
cluster_min_nodes = 1
cluster_max_nodes = 3
vm_size = "STANDARD_NC6" #NC6 is GPU-enabled
      
cts = ws.compute_targets
if batchai_cluster_name in cts:
    found = True
    print('Found existing compute target...%s' % batchai_cluster_name)
    compute_target = cts[batchai_cluster_name]
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size, 
                                                               # vm_priority = 'lowpriority', #optional
                                                                min_nodes = cluster_min_nodes, 
                                                                max_nodes = cluster_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, batchai_cluster_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    # For a more detailed view of current BatchAI cluster status, use the 'status' property    
    compute_target.status.serialize()

# Create a regression model

## Configure run settings

Create a DataReferenceConfiguration object to inform the system what data folder to download to the compute target. The path_on_compute should be an absolute path to ensure that the data files are downloaded only once.  The get_data method should use the same path to access the data files.

### Setup DataReference

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.core.runconfig import DataReferenceConfiguration

dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore=AZURE_IOT_HUB_NAME, 
                   path_on_compute='/tmp/azureml_runs',
                   mode='download', # download files from datastore to compute target
                   overwrite=False)

### Update run settings

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to the Azure ML managed compute
conda_run_config.target = compute_target

# set the data reference of the run coonfiguration
conda_run_config.data_references = {ds.name: dr}

#specify package dependencies needed to load data and train the model
cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'fastavro'], conda_packages=['numpy'])
conda_run_config.environment.python.conda_dependencies = cd

## Create data retrieval script

Remote execution requires a .py file containing a get_data() function that will be used to retrieve data from the mounted storage.  We will create the file for retrieving data by copying the utils.py file to our script folder as get_data.py.  Then we will append a get_data(), which uses the utility methods for data loading, into the newly created get_data.py.  

### Create a directory 

In [ ]:
import os
script_folder = './turbofan-regression'
os.makedirs(script_folder, exist_ok=True)

### Create get data script

In [ ]:
# create the script by copyting utils.py to the script_folder
import shutil
shutil.copyfile('utils.py', script_folder + '/get_data.py')

In [ ]:
# append the get_data method to the newly created get_data.py
%%writefile -a $script_folder/get_data.py

def get_data():
    #for the sake of simplicity use all sensors as training features for the model
    features = ["Sensor"+str(i) for i in range(1,22)]
    train_pd = load_avro_directory('/tmp/azureml_runs/**/*')
    train_pd = add_rul(train_pd)
    y_train = train_pd['RUL'].values
    X_train = train_pd[features].values
    return { "X" : X_train, "y" : y_train}



## Run the experiment on Azure ML compute 
### Instantiate AutoML

In the interest of time, the cell below uses a short iteration timeout, **1 min**, and a small number of iterations, **10**. Longer iteration timeouts and a greater number of iterations will yield better results

In [ ]:
import logging
from azureml.train.automl import AutoMLConfig

#name project folder and experiment
experiment_name = 'turbofan-regression-remote'

automl_settings = {
    "iteration_timeout_minutes": 1,
    "iterations": 10,
    "n_cross_validations": 10,
    "primary_metric": 'spearman_correlation',
    "max_cores_per_iteration": -1,
    "enable_ensembling": True,
    "ensemble_iterations": 5,
    "verbosity": logging.INFO,
    "preprocess": True,
    "enable_tf": True,
    "auto_blacklist": True
}

Automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'auto-regress.log',
                             path=script_folder,
                             run_configuration=conda_run_config,
                             data_script = script_folder + "/get_data.py",
                             **automl_settings
                            )

### Run the experiment

Run the experiment on the remote compute target and show results as the runs execute. Assuming you have kept the auto_ml settings set in the notebook this step will take several minutes.  If you have increased the number of iterations of the iteration timeout it will take longer.

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment=Experiment(ws, experiment_name)
regression_run = experiment.submit(Automl_config, show_output=False)
RunDetails(regression_run).show()

# Explore the results

Explore the results of the automatic training using the run details.

## Reconstitute a run
Given the long running nature of running the experiment the notebook may have been closed or timed out.  In that case, to retrieve the run from the run id set the value of `run_id` to the run_id of the experiment. We use `AutoMLRun` from `azureml.train.automl.run`

In [6]:
from azureml.train.automl.run import AutoMLRun
from azureml.core.experiment import Experiment
from azureml.core import Workspace

run_id = 'AutoML_xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'

if 'regression_run' not in globals():
    ws = Workspace.from_config()
    experiment_name = 'turbofan-regression-remote'
    experiment=Experiment(ws, experiment_name)
    regression_run = AutoMLRun(experiment = experiment, 
                               run_id = run_id)

regression_run.id

'AutoML_e1d6a076-6b98-4222-a0c1-acdeb861c9e8'

## Retrieve all iterations

View the experiment history and see individual metrics for each iteration run.

In [7]:
children = list(regression_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

import pandas as pd
rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

,0,1,2,3,4,5,6,7,8,9
explained_variance,0.598114,0.390203,0.337219,0.538539,0.556496,0.590431,0.569029,0.587016,0.313204,0.597787
mean_absolute_error,41.158197,55.180982,57.217501,45.047540,44.289862,42.028387,43.595751,42.269064,58.244588,41.221683
median_absolute_error,29.480241,47.278671,48.135341,33.973801,33.622797,31.027954,32.487259,31.280734,49.154011,29.660043
normalized_mean_absolute_error,0.075938,0.101810,0.105567,0.083114,0.081716,0.077543,0.080435,0.077987,0.107462,0.076055
normalized_median_absolute_error,0.054392,0.087230,0.088811,0.062682,0.062035,0.057247,0.059940,0.057714,0.090690,0.054723
normalized_root_mean_squared_error,0.105218,0.129355,0.134858,0.112526,0.110313,0.106007,0.108745,0.106449,0.137277,0.105187
normalized_root_mean_squared_log_error,NaN,0.133442,NaN,0.091530,NaN,NaN,0.094345,NaN,0.140821,NaN
r2_score,0.596469,0.390106,0.337128,0.538474,0.556447,0.590382,0.568976,0.586969,0.313132,0.596707
root_mean_squared_error,57.028303,70.110482,73.092989,60.988988,59.789604,57.456003,58.939643,57.695387,74.404381,57.011321
root_mean_squared_log_error,NaN,0.840299,NaN,0.576376,NaN,NaN,0.594103,NaN,0.886766,NaN


## Register the best model 

Use the `regression_run` object to get the best model and register it into the workspace. 

In [8]:
best_run, fitted_model = regression_run.get_output()

# register model in workspace
description = 'Aml Model ' + regression_run.id[7:15]
tags = None
regression_run.register_model(description=description, tags=tags)
regression_run.model_id # Use this id to deploy the model as a web service in Azure

Registering model AutoMLe1d6a0766best


'AutoMLe1d6a0766best'

### Save model information for deployment

Persist the information that we will need to deploy the model in the [turbofan deploy model](./turbofan_deploy_model.ipynb)

In [19]:
import json
import os

model_information = {'regressionRunId': regression_run.id, 'modelId': regression_run.model_id, 'experimentName': experiment.name}
with open('./aml_config/model_config.json', 'w') as fo:
  json.dump(model_information, fo)

# Load test data

In the test set, the time series ends some time prior to system failure. The actual
remaining useful life (RUL) are given in the RUL_*.txt files.  The data in the RUL files is a single vector where the index corresponds to the unit number of the engine and the value corresponds to the actual RUL at the end of the test.

The RUL for a given cycle in the training set is given by adding the RUL at test end (from the RUL vector file) to the maximum cycle in the test data and then subtracting the current cycle:

$$RUL_{current} =  RUL_{TestEnd} + Cycle_{max} - Cycle_{current}$$

Taking unit number 1 as an example:
   * Taking the first value from RUL_FD003.txt gives:       $RUL_{TestEnd} = 44$
   * The final(max) cycle value from test_FD003.txt gives:  $Cycle_{max} = 233$
   * The values for the first 5 cycles for engine 1 are:

|Unit|Cycle|Max Cycle|Test End RUL|Remaining Life|
|-----|-----|-----|-----|-----|
|1|1|233|44|276|
|1|2|233|44|275|
|1|3|233|44|274|
|1|4|233|44|273|
|1|5|233|44|272|



## Define some methods for loading from text files

In [27]:
import json
import utils
import pandas as pd
from os.path import isfile

def add_column_names(data_frame):
    data_frame.columns = (["Unit","CycleTime"]
                          + ["OperationalSetting"+str(i) for i in range(1,4)]
                          + ["Sensor"+str(i) for i in range(1,22)])

def read_data_file(full_file_name):
    data = pd.read_csv(full_file_name, sep = ' ', header = None)
    data.dropna(axis='columns', inplace=True)
    return data

def load_rul_data(full_file_name):
    rul_data = read_data_file(full_file_name)

    # add a column for the unit and fill with numbers 1..n where
    # n = number of rows of RUL data
    rul_data['Unit'] = list(range(1, len(rul_data) + 1))
    rul_data.rename(columns = {0 : 'TestEndRUL'}, inplace = True)
    return rul_data


def load_test_data(test_full_file_name, rul_full_file_name):
    data = read_data_file(test_full_file_name)
    add_column_names(data)
    data = utils.add_maxcycle(data)
    
    rul_data = load_rul_data(rul_full_file_name)
    data = data.merge(rul_data, how = 'left', left_on = 'Unit', right_on = 'Unit')
    data['RUL'] = data.apply(lambda r: int(r['MaxCycle'] + r['TestEndRUL'] - r['CycleTime']), axis = 1)
    data['RulClass'] = data.apply(lambda r: utils.classify_rul(r['RUL']), axis = 1)

    return data


## Read and process the test data

><font color=gray>_Note: the dataset should correspond to the dataset from deviceharness_</font>


In [28]:
dataset = "FD004" 
rul_file_name = 'data/RUL_' + dataset + '.txt'
test_file_name = 'data/test_' + dataset + '.txt'

test_pd = load_test_data(test_file_name, rul_file_name)
test_pd.head(5)

,Unit,CycleTime,OperationalSetting1,OperationalSetting2,OperationalSetting3,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,...,Sensor16,Sensor17,Sensor18,Sensor19,Sensor20,Sensor21,MaxCycle,TestEndRUL,RUL,RulClass
0,1,1,20.0072,0.7000,100.0,491.19,606.67,1481.04,1227.81,9.35,...,0.02,362,2324,100.00,24.31,14.7007,230,22,251,Full
1,1,2,24.9984,0.6200,60.0,462.54,536.22,1256.17,1031.48,7.05,...,0.02,306,1915,84.93,14.36,8.5748,230,22,250,Full
2,1,3,42.0000,0.8420,100.0,445.00,549.23,1340.13,1105.88,3.91,...,0.02,328,2212,100.00,10.39,6.4365,230,22,249,Full
3,1,4,42.0035,0.8402,100.0,445.00,549.19,1339.70,1107.26,3.91,...,0.02,328,2212,100.00,10.56,6.2367,230,22,248,Full
4,1,5,35.0079,0.8400,100.0,449.44,555.10,1353.04,1117.80,5.48,...,0.02,333,2223,100.00,14.85,8.9326,230,22,247,Full


### Serialize test data

Save off the data so that we can use it when we test the web service in the [turbofan deploy model](./turbofan_deploy_model.ipynb) notebook.

In [30]:
test_pd.to_csv('./data/WebServiceTest.csv')

# Test regression model

predict on training and test set and calculate residual values

In [ ]:
selected_features = ["Sensor"+str(i) for i in range(1,22)]

#load the values used to train the model
X_train = train_pd[selected_features].values
y_train = train_pd['RUL'].values

#predict and calculate residual values for train
y_pred_train = fitted_model.predict(X_train)
y_residual_train = y_train - y_pred_train

train_pd['predicted'] = y_pred_train; 
train_pd['residual'] = y_residual_train

#load the values from the test set
X_test = test_pd[selected_features].values
y_test = test_pd['RUL'].values

#predict and calculate residual values for test
y_pred_test = fitted_model.predict(X_test)
y_residual_test = y_test - y_pred_test

test_pd['predicted'] = y_pred_test;
test_pd['residual'] = y_residual_test

In [ ]:
train_pd[['Unit', 'RUL', 'predicted', 'residual']].head(5)
test_pd[['Unit', 'RUL', 'predicted', 'residual']].head(5)

## Predicted vs. actual

Plot the predicted RUL against the actual RUL.  The dashed line represents the ideal model.

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

fig, (ax1,ax2) = plt.subplots(nrows=2, sharex=True)
fig.set_size_inches(16, 16)

font_size = 14

g = sns.regplot(y='predicted', x='RUL', data=train_pd, fit_reg=False, ax=ax1)
lim_set = g.set(ylim=(0, 500), xlim=(0, 500))
plot = g.axes.plot([0, 500], [0, 500], c=".3", ls="--");

rmse = ax1.text(16,450,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_train, y_pred_train))), fontsize = font_size)
r2 = ax1.text(16,425,'R2 Score = {0:.2f}'.format(r2_score(y_train, y_pred_train)), fontsize = font_size)

g2 = sns.regplot(y='predicted', x='RUL', data=test_pd, fit_reg=False, ax=ax2)
lim_set = g2.set(ylim=(0, 500), xlim=(0, 500))
plot = g2.axes.plot([0, 500], [0, 500], c=".3", ls="--");

rmse = ax2.text(16,450,'RMSE = {0:.2f}'.format(np.sqrt(mean_squared_error(y_test, y_pred_test))), fontsize = font_size)
r2 = ax2.text(16,425,'R2 Score = {0:.2f}'.format(r2_score(y_test, y_pred_test)), fontsize = font_size)

ptitle = ax1.set_title('Train data', size=font_size)
xlabel = ax1.set_xlabel('Actual RUL', size=font_size)
ylabel = ax1.set_ylabel('Predicted RUL', size=font_size)

ptitle = ax2.set_title("Test data", size=font_size)
xlabel = ax2.set_xlabel('Actual RUL', size=font_size)
ylabel = ax2.set_ylabel('Predicted RUL', size=font_size)

## Predicted vs. residual

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=2, sharex=True)
fig.set_size_inches(16, 16)

font_size = 14

g = sns.regplot(y='residual', x='predicted', data=train_pd, fit_reg=False, ax=ax1)
lim_set = g.set(ylim=(-350, 350), xlim=(0, 350))
plot = g.axes.plot([0, 350], [0, 0], c=".3", ls="--");

g2 = sns.regplot(y='residual', x='predicted', data=test_pd, fit_reg=False, ax=ax2)
lim_set = g2.set(ylim=(-350, 350), xlim=(0, 350))
plot = g2.axes.plot([0, 350], [0, 0], c=".3", ls="--");

ptitle = ax1.set_title('Train data', size=font_size)
xlabel = ax1.set_xlabel('Predicted RUL', size=font_size)
ylabel = ax1.set_ylabel('Residual', size=font_size)

ptitle = ax2.set_title("Test data", size=font_size)
xlabel = ax2.set_xlabel('Predicted RUL', size=font_size)
ylabel = ax2.set_ylabel('Residual', size=font_size)

## Residual distribution

Plot histogram and Q-Q plot for test and train data to check for normal distibution of residuals

In [ ]:
import statsmodels.api as sm
import scipy.stats as stats

fig, (ax1,ax2) = plt.subplots(nrows=2, ncols= 2)
fig.set_size_inches(16, 16)

g = sns.distplot(train_pd['residual'], ax=ax1[0], kde=False)
g = stats.probplot(train_pd['residual'], plot=ax1[1])

g2 = sns.distplot(test_pd['residual'], ax=ax2[0], kde=False)
g2 = stats.probplot(test_pd['residual'], plot=ax2[1])


ptitle = ax1[0].set_title('Residual Histogram Train', size=font_size)
xlabel = ax1[0].set_xlabel('Residuals', size=font_size)
ptitle = ax1[1].set_title('Q-Q Plot Train Residuals', size=font_size)

ptitle = ax2[0].set_title('Residual Histogram Test', size=font_size)
xlabel = ax2[0].set_xlabel('Residuals', size=font_size)
ptitle = ax2[1].set_title('Q-Q Plot Test Residuals', size=font_size)



## Score model

The PHM08 scoring method is used to derive a score for the model. The method penalizes low predictions (i.e. negative residuals) more heavily than high predictions. The score only takes into account the models prediction of RUL for the last cycle of the test set.

In [ ]:
def score(x):
    diff = x.predicted-x.RUL
    result = np.expm1(diff/-10.) if diff < 0. else np.expm1(diff/13.)
    return result

max_unit = test_pd.groupby(['Unit']).agg({'CycleTime' : 'max'})
max_unit['Unit'] = list(range(1, len(max_unit) + 1))

max_unit = pd.merge(max_unit, test_pd, on=['CycleTime', 'Unit'])

max_unit["score"] = max_unit.apply(score, axis=1)
score = sum(max_unit.score)
round(score)